In [1]:
import cv2
import numpy as np

def get_four_points(image_path):
    # Load the image
    if type(image_path) == str:
        img = cv2.imread(image_path)
    else:
        img = image_path

    # Create a window to display the image
    cv2.namedWindow('Image')
    
    # Initialize a list to store the four points
    points = []

    def click_event(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            # Append the clicked point to the list
            points.append((x, y))
            # Draw a circle at the clicked point on the image
            cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
            # Display the image with the clicked point
            cv2.imshow('Image', img)

    # Set the mouse callback function to handle mouse events
    cv2.setMouseCallback('Image', click_event)

    # Display the image and wait for the user to click four points
    cv2.imshow('Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

    # Convert the list of points to a NumPy array and return it
    return np.array(points, dtype=np.float32)

def get_position(df, frame_id): 
    filtered_df = df[df['frame_id'] == frame_id]

# Extract values from columns x_center and y_center and put them in one NumPy array
    position_array = filtered_df[['x_center', 'y_center']].values.tolist()
    labels = filtered_df[['label']].values
    colors = filtered_df[['color']].values

    return position_array, labels, colors

    

In [3]:
import cv2 
import numpy as np 
import pandas as pd
cap = cv2.VideoCapture("/Users/menghang/Downloads/kaggle/working/football-server/YOLOX_outputs/yolox_m_mix_det/track_vis/2023_08_04_04_58_10/Film Role-0 ID-3 T-2 m00s00-000-m00s00-185.avi")
_,frame = cap.read()

corners_pitch = get_four_points(frame)


template = cv2.imread("/Users/menghang/Desktop/ml-dev/football-server/tracker/inference/black.jpg")
template_corners = get_four_points("/Users/menghang/Desktop/ml-dev/football-server/tracker/inference/black.jpg")

M, _ = cv2.findHomography(corners_pitch, template_corners)

frame_id = 0 

df = pd.read_csv("/Users/menghang/Desktop/ml-dev/football-server/YOLOX_outputs/yolox_m_mix_det/track_vis/2023_08_04_04_58_10.txt",names=["frame_id", "id", "x", "y", "w", "h", "score", "label", "x_center", "y_center", "color"])

while cap.isOpened(): 
    success, frame = cap.read()
    temp = template.copy()
    if success: 
        players_position_2d, labels, colors = get_position(df, frame_id )
        players_position_2d = np.array(players_position_2d, dtype=np.float32).reshape(-1, 1, 2)
        players_position_2d = cv2.perspectiveTransform(players_position_2d , M)
        if(not isinstance(players_position_2d, np.ndarray)):
           continue
        for player_position, label, color in zip(players_position_2d, labels, colors) :
          x, y = player_position[0]
          color = color[0][:color[0].index(".")]
          print(color)
          if label[0] == "ball":
            cv2.circle(temp,  (int(x),int(y)), 5, (255, 192, 203	), -1)
            
          else: 
            if color == "blue" or color == "cyan" or color == "y":
                cv2.circle(temp,  (int(x),int(y)), 10, (255,0,0), -1)
            elif color == "red":
                cv2.circle(temp,  (int(x),int(y)), 10, (0,0,255	), -1)
            elif color == "white" or color == "b":
                cv2.circle(temp,  (int(x),int(y)), 10, (255, 255,255	), -1)

            
          
        cv2.imshow("frame", temp)
        frame_id += 1

                            #                bg_ratio + 1, (0, 255, 255), -1)
        if cv2.waitKey(25) & 0xFF == ord('q'):
          cv2.destroyAllWindows()
          cv2.waitKey(1) 
          break
    else:
        break


cap.release()

    
        





    

white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
white
y
blue
blue
red
cyan
cyan
y
cyan
white
b
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
b
white
blue
blue
blue
red
cyan
cyan
y
cyan
white
b
white
blue
blue
cyan
red
cyan
cyan
blue
cyan
white
blue
white
blue
blue
cyan
red
cyan
cyan
blue
cyan
white
blue
white
blue
blue
cyan
red
cyan
cyan
blue
cyan
white
blue
white
blue
blue
cyan
red
cyan
cyan
blue
cyan
white
blue
white
blue
blue
cyan
red
cyan
white
blue
cyan
white
blue
white
blue
blue
cyan
red
cyan
white
blue
cya

In [13]:
import cv2
from sklearn.cluster import KMeans
import numpy as np
color_for_labels = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

def compute_color_for_labels(label):
    """
    Simple function that adds fixed color depending on the class
    """
    color = [int((p * (label ** 2 - label + 1)) % 255)
             for p in color_for_labels]
    return tuple(color)
pallete = {
           'green': (0, 128, 0),
           'red': (200, 16, 46),
           'white': (255, 255, 255),
           'blue': (213, 76,41),

}
def closest_color(list_of_colors, color):
    colors = np.array(list_of_colors)
    color = np.array(color)
    distances = np.sqrt(np.sum((colors-color)**2, axis=1))
    index_of_shortest = np.where(distances == np.amin(distances))
    shortest_distance = colors[index_of_shortest]

    return shortest_distance

def find_key(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None


def detect_color(img):
    # img = background_subtraction(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.reshape((img.shape[1]*img.shape[0], 3))

    kmeans = KMeans(n_clusters=2)
    s = kmeans.fit(img)

    labels = kmeans.labels_
    centroid = kmeans.cluster_centers_
    labels = list(labels)
    percent = []

    for i in range(len(centroid)):
        j = labels.count(i)
        j = j/(len(labels))
        percent.append(j)

    detected_color = centroid[np.argmin(percent)]

    list_of_colors = list(pallete.values())
    assigned_color = closest_color(list_of_colors, detected_color)[0]
    rgb_assigned_color = (int(assigned_color[0]), int(
        assigned_color[1]), int(assigned_color[2]))

    assigned_color = (int(assigned_color[2]), int(
        assigned_color[1]), int(assigned_color[0]))
    if assigned_color == (0, 0, 0):
        assigned_color = (128, 128, 128)
    k = find_key(pallete, rgb_assigned_color)

    return assigned_color, k

In [14]:
def get_bbox(df, frame_id): 
    filtered_df = df[df['frame_id'] == frame_id]

# Extract values from columns x_center and y_center and put them in one NumPy array
    position_array = filtered_df[['x', 'y', 'w', 'h']].values.tolist()


    return position_array

In [17]:
import cv2 
import numpy as np 
import pandas as pd
cap = cv2.VideoCapture("/Users/menghang/Downloads/kaggle/working/football-server/YOLOX_outputs/yolox_m_mix_det/track_vis/2023_08_04_04_58_10/Film Role-0 ID-3 T-2 m00s00-000-m00s00-185.avi")
_,frame = cap.read()


frame_id = 0 

df = pd.read_csv("/Users/menghang/Desktop/ml-dev/football-server/YOLOX_outputs/yolox_m_mix_det/track_vis/2023_08_04_04_58_10.txt",names=["frame_id", "id", "x", "y", "w", "h", "score", "label", "x_center", "y_center", "color"])

while cap.isOpened(): 
    success, frame = cap.read()
    if success: 
        bboxes = get_bbox(df, frame_id )
        for box in bboxes:
           

            x,y,w,h = box
            x,y,w,h = int(x), int(y), int(w), int(h)
            f = frame[y:y+h, x:x+w]
            _, k = detect_color(f)
            
        frame_id += 1
        print(frame_id)
                            #                bg_ratio + 1, (0, 255, 255), -1)
        if cv2.waitKey(25) & 0xFF == ord('q'):
          cv2.destroyAllWindows()
          cv2.waitKey(1) 
          break
    else:
        break


cap.release()

    
        





    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


KeyboardInterrupt: 

In [19]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('/Users/menghang/Desktop/Screenshot 2023-08-04 at 12.51.14 AM.png')

# Convert to HSV color space
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define color ranges for blue and white jerseys (example values)
lower_blue = np.array([90, 100, 100])
upper_blue = np.array([120, 255, 255])

lower_white = np.array([0, 0, 150])
upper_white = np.array([180, 50, 255])

# Create masks for blue and white jerseys
blue_mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
white_mask = cv2.inRange(hsv_image, lower_white, upper_white)

# Find contours for blue and white jerseys
contours_blue, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_white, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes around blue jerseys
for contour in contours_blue:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Draw bounding boxes around white jerseys
for contour in contours_white:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 255), 2)

# Display the result
cv2.imshow('Jersey Color Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1